In [1]:
import cocoex
import pickle
import pandas as pd
import numpy as np

In [2]:
algnames = ["pso", "ata", "gsa", "ffa", "rio", "de", "saa"]

restarts    = 5
repetitions = 3
functions  = ",".join(filter(None, [str(i) if i not in [1,6,10, 16,23] else None for i in range(1,25)]))

dimensions = 5
indices    = 1
precision = 1e-8

In [3]:
### all parameter setting, one repetition ###

#bbob
suite_string  = f"function_indices:{functions} dimensions:{dimensions} instance_indices:{indices}"
suite       = cocoex.Suite("bbob", "", suite_string)
problemnames = [problem.id for problem in suite]

display(suite)

#target
file = open("bbob_final_target_fvalue1.pkl",'rb')
d = pickle.load(file)
file.close()

Suite(b'bbob', b'', b'function_indices:2,3,4,5,7,8,9,11,12,13,14,15,17,18,19,20,21,22,24 dimensions:5 instance_indices:1')

In [4]:
# compute error to the optimum
def get_error(filea, fileb, repetitions, restarts, target, precision):
    result = pd.DataFrame([])
    for rep in range(repetitions):
        best_fitness = []
        for res in range(restarts):
            infile = f"{a}res{res}_rep{rep}{b}"
            df = pd.read_csv(infile, header=None, names=["nfe","fvalue","xvalue"])

            error = df.fvalue.min()-target
            error = 0 if error <= precision else error
            
            best_fitness += [error]

        result[rep] = best_fitness
    return result.mean(axis=1)

df_all = [pd.DataFrame([], index=problemnames, columns=algnames) for _ in range(restarts)]

b = ".csv"
for alg in algnames:
    #success_par_tmp = pd.DataFrame([], columns=[problemnames])
    for problem in suite:
        #print(f"processing {alg}:{problem.id}")
        a = f"results/{alg}_000/{problem.id}_"
        
        r = get_error(filea=a, fileb=b, 
                      repetitions=repetitions, restarts=restarts,
                        target=d[problem.id], precision=precision)
        
        for i in range(restarts):
            df_all[i].loc[[problem.id],[alg]] = [r[i]]

df_all[0].head()

,pso,ata,gsa,ffa,rio,de,saa
bbob_f002_i01_d05,229.138067,0.006315,8.637944,1.175029,111076.551198,0.0,1.870975
bbob_f003_i01_d05,0.994959,2.321569,33.733709,29.517715,51.244804,0.00017,34.587956
bbob_f004_i01_d05,3.316529,9.949574,52.282741,66.993507,89.500002,0.000046,16.763946
bbob_f005_i01_d05,0.0,0.0,0.0,0.0,0.0,0.0,0.0
bbob_f007_i01_d05,0.305609,1.224868,2.447393,0.180438,21.977494,0.0,4.124699


In [5]:
# join set of df into one df
colnames = [f"{alg}{r}" for alg in algnames for r in range(restarts)]
df_join = pd.DataFrame([], index=problemnames, columns=colnames)

for alg in algnames:
    for r in range(restarts):
        df_join[f"{alg}{r}"] = df_all[r][f"{alg}"]
    

df_join.head()

,pso0,pso1,pso2,pso3,pso4,ata0,ata1,ata2,ata3,ata4,...,de0,de1,de2,de3,de4,saa0,saa1,saa2,saa3,saa4
bbob_f002_i01_d05,229.138067,0.0,0.0,0.0,0.0,0.006315,0.0,0.0,0.00642,0.0,...,0.0,0.0,0.0,0.0,0.0,1.870975,1.239555,5.298547,5.298547,1.870975
bbob_f003_i01_d05,0.994959,0.994959,1.326612,0.0,1.658265,2.321569,1.658265,0.331653,2.321571,0.994959,...,0.00017,0.0,0.0,0.0,0.0,34.587956,19.921005,3.788441,3.788441,34.587956
bbob_f004_i01_d05,3.316529,4.97479,2.653222,0.0,5.969744,9.949574,2.653222,1.658265,5.306447,2.653222,...,0.000046,0.0,0.331653,0.0,0.000004,16.763946,4.892345,2.214184,2.214184,16.763946
bbob_f005_i01_d05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
bbob_f007_i01_d05,0.305609,0.0,0.619558,0.0,0.002433,1.224868,0.155237,0.305609,0.21339,0.004866,...,0.0,0.0,0.0,0.0,0.002433,4.124699,1.230209,0.160682,0.160682,4.124699


In [6]:
df_join.to_csv("performance_profiles.csv")